# Regression II

Continuiamo con la regression.

### Regularization

Bisogna fare attenzione con la regression: quando aumenti la complessità del tuo modello, rischi di fare **overfitting** dei dati. Di conseguenza, aumenti la varianza del modello -> poor cross-validation and dataset testing performance.

Per i casi in cui sei "concerned" con l'overfitting, al posto di fare la log-likelihood legata esclusivamente alla somma dei quadrati ( vedi notebook ) puoi **additional constraints**:

* smoothness
* number of coefficients
* size of coefficients

Introduci quindi un **regularization parameter**, che indichiamo con $\lambda$ nelle note.

La **modified log-likelihood** viene massimizzata, o minimizzi qualcos'altro (?): a questo punto arrivi ad una soluzione per i tuoi parametri, $\hat{\theta}$, che trovi sul notebook.

Dal punto di vista *bayesiano* hai applicato una prior ( in questo caso, gaussiana ), per dare dei constraints ai tuoi parametri => controlli la posterior.

> La regularization prova a portarti nel "regime" in cui hai una soluzione *bilanciata* tra overfitting e underfitting.

#### Ridge Regression

Una prior gaussiana sui coefficienti di regressione corrisponde alla cosiddetta **Ridge Regression** // Tikhonov Regularization, che *penalizza i coefficienti di regressione* con $\theta^{T}\theta \equiv \sum_{i} \theta^{2}_{i} < s$, dove $s$ è un **tolerance value** che definiremo a breve.

=> Sopprimo i coefficienti di regressione "grandi" <br>
=> Limito la varianza del mio modello <br>
=> Aumento il bias - è il lato negativo della cosa

#### LASSO: Least Absolute Shrinkage and Selection Regularization

In questo caso applico un constraint diverso: vedi il notebook per la formula precisa, ma è l'equivalente di minimizzare i least-squares con la condizione $\sum_{i} |\theta_{i}| s$ . 

=> penalizzo la somma dei valori assoluti dei coefficienti <br>
=> impongo **sparsity** sul regression model: preferisco regioni, nello spazio della likelihood, che corrispondono ai vertici della regione definita dalla regularization (vedi disegno poco sopra nel notebook)
=> vertice? -> setti alcuni model attributes to zero

Guarda cosa succede nel notebook: scelgo solo *alcune* delle CENTO gaussiane con cui *potrei* fittare. *La scelta è data-driven*!

#### Constraint Term

Scrivo $s > \lambda \sum_{i} |\theta^{i}|^{q}$:

=> $q=2$ è la Ridge Regression <br>
=> $q=1$ è la LASSO Regression

Pensa ad $s$ come ad una distanza normalizzata. $s=1$ significa NON imporre costraints. $s<1$ limita i $\theta_{i}$.

**Ridge e LASSO sono bayesian estimates with different priors. La differenza sta nella FORMA della constraint region.**

---

##### Note 

Setting $\lambda=0$ is mathemathicall identical to no regularizaion. But that's not necessarily true in the scikit-learn implementation: i.e. Ridge and Lasso with alpha=0 might not give the same result of LinearRegression. The regularization algorithms have additional sophistications to improve convergence.

---

#### Elastic Net, Deep Learning

La regularization è **essenziale** in deep learning. L'elastic net strategy combina Ridge e Lasso per "salvarti" dall'overfitting.

#### Come scelgo $\lambda$?

Usi la cross-validation! Ci sono pacchetti di *sklearn* che lo fanno apposta: `RidgeCV` e `LassoCV` sono quelli da guardare.

---

Guarda il notebook per l'esempio sul Boston Housing Data

---

### Locally Linear Regression: LOWESS / LOESS

In questo tipo di regression assumiamo che la regression function può essere approssimata in ogni punto con un'espansione in serie di Taylor: se tronco al primo termine, trovo la Kernel Regression.

La LLR è simile a quanto appena detto, ma fitto la local regression ai punti pesati trovando una *weighting function* w(x) che minimizza la quantità segnata sul notebook.

LOWESS: Locally Weighted Scatter Plot Smoothing - usa "tricubic" kernel. Come vedi dall'espressione di $K$, vedi che è una via di mezzo tra qualcosa di lineare e qualcosa di non lineare.

LOESS: LOcal regrESSion.

**Non approfondiamo questa roba, tieni solo presente che esistono vie di mezzo tra linear e non-linear regression.**

---

### Non-Linear Regression

Spesso posso linearizzare i miei dati non-lineari, ma ho delle conseguenze: ad esempio, le error bars diventano asimmetriche. Posso considerare una **non-linear regression**, in cui il modello dipende non-linearmente dai parametri di regressione.

<span style="color:red"> *Se sai la forma teorica del modello*, puoi usare MCMC e campionare lo spazio dei parametri alla ricerca dei valori ottimali. Questo lo sai fare. Un altro approccio, è usare il **Levenberg-Marquardt (LM) Algorithm** per ottimizzare la MLE. **QUESTA ROBA È LETTERALMENTE OVUNQUE, QUINDI STUDIALA BENE** </span>

Per ora diciamo solo cosa fa: "LM searches through a combination of **gradient descent** and **Gauss-Newton optimization**.

Se posso scrivere la mia regression function come espansione in serie di taylor, allora al primo ordine posso scrivere: 

$$f(x_{i}|\theta) = f(x_{i}|\theta_{0}) + J d\theta$$

dove $\theta_{0}$ è la mia initial guess per i regression parameters, $J$ è la jacobiana valutata attorno al punto ($J = \partial f(x_{i}|\theta) \ / \ \partial \theta$) e $d\theta$ è una perturbazione nei parametri di regressione.

La LM minimizza la somma degli errori al quadrato per una perturbazione $d\theta$ (vedi formula sul notebook). Trovo quindi una relazione *iterativa* (#) per $d\theta$ data dalla formula sul notebook, in cui $\lambda$ si comporta come un **damping parameter**.

* se $\lambda$ è piccolo, allora la relazione (#) approssima un Gauss-Newton method -> minimizzo i parametri assumendo funzione quadratica
* se $\lambda$ è grande, $d\theta$ segue la direzione della discesa più ripida
* il termine diag($J^{T}C^{-1}J$) è quello che lo rende diverso dalla Ridge Regression, e ti assicura che l'aggiornare iterativamente $d\theta$ sia massimo lungo le direzioni in cui il gradiente è minimo -> miglioro la convergenza

**il processo iterativo termina quando i cambiamenti nella likelihood raggiungono un limite predeterminato.**

"Gradient Descent"? -> ne riparliamo nella parte di neural networks.

> Sia `scipy.optimize.curve_fit` che `scipy.optimize.leastsq` implementano l'algoritmo LM. L'ho usato più spesso di quanto pensassi, e la cosa divertente è che intuitivamente ho pensato esattamente a lui guardando questa roba, LOL.

### Gaussian Process Regression (GPR)

È una (potente) classe di algoritmi di regressione. È ampiamente applicabile anche a dati NON generati da processo gaussiano; pul dare modelli molto generali e flessibili, che sono MORE data-driven di altri.

**Non solo la GPR è un'ottima data-driven interpolation technique, ma ti dà anche una misura dell'incertezza di interpolazione**

Guarda l'esempio sul notebook per il lavoro di fit che fa Gerosa. Diamo una definizione precisa:

> A Gaussian Process (GP) is a collection of random variables in a parameter space for which any subset can be defined by a joint Gaussian distribution.


<span style="color:blue"> Devo pensarci un po' su... </span>

Guarda sul notebook gli esempi che sono illustrati nel notebook.

*h* è la bandwidth: per una given bandwidth, puoi definire un infinito set di **covariance functions**. PUOI DEFINIRNE DIVERSE PER INTERPOLARE I DATI.

Per la GP regression, vogliamo stimare valore e varianza di un nuovo dato di punti dato un data set in input. Questo equivale a *mediare su tutte le funzioni che passano nei dati in input*.

Calcoliamo la matrice di covarianza di tutti i dati osservati e il nuovo set di punti che voglio prevedere: $K$ è indicata sul notebook. L'indice 1 è relativo ai punti in input, l'indice 2 è relativo ai punti sconosciuti.

Si può mostrare, che per due vettori osservati x ed y, e un vettore di punti ignoti x*, la posterior è data da $\mathcal{N}(\vec{\mu}, \sigma)$, con le definizioni in slide. 

* $\mu_{j}$ dà l'expected value $f_{j}^{*}$ del risultato previsto
* $\sigma_{jk}$ dà l'error covariance tra due punti sconosciuti

> Il punto sta proprio qui: so media E incertezza di un punto previsto!

Nota che la fisica del processo alla base dei miei dati ENTRA attraverso la forma ASSUNTA della funzione di covarianza!

### Uncertainties All Round

Che succede nel mondo reale, dove anche la variabile indipendente *x* non è libera da incertezze?

Guarda il notebook. La relazione lineare "vera" è lineare; misuriamo x ed y, con rumore gaussiano. Risolvo per y e trovo la relazione illustrata. Quindi $\hat{y}$ dipende dal rumore in x, e l'incertezza in x è ora parte della regression equation e scala coi regression coefficients.

Questo problema si chiama **Total Least Squares (TLS)**. In TLS, costruisco una likelihood che misura la *perpendicular distance* dei punti da una linea modellata, rispetto a quel che ho fatto finora coi least squares, dove misuro solo la *vertical distance* (vedi notebook).

Come tengo presente le incertezze di misura in entrambe le variabili? Assumo che siano gaussiane -> ho una total data covariance matrix $\sigma_{i}$.

Consideriamo la straight-line regression. Definisco la nostra linea con un'interpretazione geometrica, in termini del vettore normale $n$. 

La matrice di covarianza è proiettata in $S^{2}_{i}$ dal vettore normale; la distanza tra il punto i-esimo e la linea è data dalla formula indicata per $\Delta_{i}$.

Scrivo la log-likelihood: $ln \mathcal{L} \propto - \sum_{i} \frac{\Delta_{i}^{2}}{2S^{2}_{i}}$ e la massimizzo con un MCMC.

Il codice nel notebook ti fa un esempio di Total Least Squares.